In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import functions as F
import os

os.environ['PYSPARK_PYTHON'] = '/usr/local/bin/python3.9'

spark = SparkSession.builder.appName("Spark_sql_test").\
    master("local[*]").\
    getOrCreate()

sc = spark.sparkContext

In [33]:
df = spark.read.format('csv').\
    option("header", "true").\
    load("car_prices.csv")
df.printSchema()
df.show()

df.createTempView("car_prices")

root
 |-- year: string (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- trim: string (nullable = true)
 |-- body: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- vin: string (nullable = true)
 |-- state: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- odometer: string (nullable = true)
 |-- color: string (nullable = true)
 |-- interior: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- mmr: string (nullable = true)
 |-- sellingprice: string (nullable = true)
 |-- saledate: string (nullable = true)

+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+---------+--------+------+--------+--------------------+-------+------------+--------------------+
|year|     make|              model|                trim|       body|transmission|              vin|state|condition|odometer| color|interior|              seller|    mmr|selling

AnalysisException: Temporary view 'car_prices' already exists

In [ ]:
# 年份统计
# show(n=30) n表示最多展示的行数
df.select('*').groupBy('year').count().sort('year',ascending=False).show(n=30)

+----+-----+
|year|count|
+----+-----+
|2015| 7994|
|2014|69712|
|2013|87467|
|2012|87380|
|2011|41384|
|2010|22616|
|2009|17959|
|2008|27011|
|2007|25378|
|2006|21631|
|2005|17169|
|2004|13624|
|2003|10368|
|2002| 7693|
|2001| 5140|
|2000| 3427|
|1999| 2227|
|1998| 1464|
|1997| 1042|
|1996|  562|
|1995|  483|
|1994|  286|
|1993|  127|
|1992|   97|
|1991|   51|
|1990|   33|
+----+-----+



In [ ]:
# make统计

df.select('*').groupBy('make').count().sort('count',ascending=False).show(n=30)

+-------------+-----+
|         make|count|
+-------------+-----+
|         Ford|81013|
|    Chevrolet|54150|
|       Nissan|44043|
|       Toyota|35313|
|        Dodge|27181|
|        Honda|24781|
|      Hyundai|18659|
|          BMW|17509|
|          Kia|15828|
|     Chrysler|15133|
|     Infiniti|14011|
|Mercedes-Benz|14008|
|         Jeep|12735|
|   Volkswagen|10780|
|        Lexus|10409|
|          GMC| 9217|
|        Mazda| 6930|
|     Cadillac| 6315|
|      Lincoln| 4926|
|         Audi| 4802|
|        Acura| 4503|
|        Buick| 4470|
|       Subaru| 4369|
|          Ram| 4044|
|      Pontiac| 4018|
|   Mitsubishi| 3661|
|        Volvo| 3082|
|         MINI| 2946|
|       Saturn| 2570|
|      Mercury| 1571|
+-------------+-----+
only showing top 30 rows



In [37]:
# 车型统计信息

sql = '''
SELECT make, model, count(*) as total
FROM car_prices
GROUP BY make, model
ORDER BY make ASC, model DESC
'''

spark.sql(sqlQuery=sql).show(n=50)

+------------+---------------+-----+
|        make|          model|total|
+------------+---------------+-----+
|       Acura|            mdx|    1|
|       Acura|            ZDX|   28|
|       Acura|TSX Sport Wagon|   28|
|       Acura|            TSX|  789|
|       Acura|            TLX|    1|
|       Acura|             TL| 1649|
|       Acura|            RSX|  130|
|       Acura|            RLX|   13|
|       Acura|             RL|   91|
|       Acura|            RDX|  300|
|       Acura|            MDX| 1236|
|       Acura|         Legend|    7|
|       Acura|        Integra|   40|
|       Acura|            ILX|  103|
|       Acura|             CL|   87|
|Aston Martin|     V8 Vantage|   16|
|Aston Martin|         Rapide|    2|
|Aston Martin|            DB9|    6|
|        Audi|allroad quattro|   30|
|        Audi|        allroad|   39|
|        Audi|            TTS|    6|
|        Audi|          TT RS|    3|
|        Audi|             TT|  109|
|        Audi|            SQ5|    6|
|

In [38]:
# 最好卖的车型top

sql = '''
SELECT make, model, count(*) as total
FROM car_prices
GROUP BY make, model
ORDER BY total DESC
'''

spark.sql(sqlQuery=sql).show(n=50)

+-------------+----------------+-----+
|         make|           model|total|
+-------------+----------------+-----+
|       Nissan|          Altima|16346|
|         Ford|          Fusion|12116|
|         Ford|           F-150|11950|
|       Toyota|           Camry|10986|
|         Ford|          Escape|10656|
|         Ford|           Focus| 9547|
|        Honda|          Accord| 8528|
|    Chevrolet|          Impala| 7500|
|          BMW|        3 Series| 7132|
|        Honda|           Civic| 7068|
|        Dodge|   Grand Caravan| 6966|
|     Infiniti|         G Sedan| 6939|
|       Toyota|         Corolla| 6745|
|         Ford|        Explorer| 6242|
|    Chevrolet|          Malibu| 6213|
|      Hyundai|          Sonata| 5787|
|    Chevrolet|  Silverado 1500| 5685|
|       Nissan|          Maxima| 5470|
|    Chevrolet|           Cruze| 5404|
|      Hyundai|         Elantra| 5343|
|         Ford|            Edge| 5098|
|     Chrysler|Town and Country| 5053|
|         Ford|         M